MongoDB with Python

In [186]:
from pymongo import MongoClient, errors, ASCENDING, DESCENDING
import bson
import time
import pytz
from datetime import datetime, timedelta


In [9]:
# init client to connect to mongoDB
my_client = MongoClient('mongodb://localhost:27017')

In [16]:
my_client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [24]:
# connect to db
my_client.list_database_names()

['SoundCloud', 'admin', 'auth_tutorial', 'config', 'local', 'store']

In [31]:
# connect to db store
db_store = my_client.store

In [26]:
db_store

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'store')

In [40]:
# list collections from store db
print(db_store.list_collection_names())

['admin', 'users']


In [41]:
users_col = db_store.users

In [42]:
users_col

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'store'), 'users')

C - Create

In [36]:
users_col.insert_one({
    "name": "John",
    "age": 29
})

In [38]:
new_user = {
    "name": "Keanu Reeves",
    "age": 57
}

In [39]:
users_col.insert_one(new_user)

In [43]:
users_col

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'store'), 'users')

R - Read

In [46]:
users_col.find_one({"name":"Keanu Reeves"})

{'_id': ObjectId('668a85a873c8b630e2131b6a'),
 'name': 'Keanu Reeves',
 'age': 57}

In [47]:
users_col.find_one(new_user)

{'_id': ObjectId('668a85a873c8b630e2131b6a'),
 'name': 'Keanu Reeves',
 'age': 57}

U- Update

In [48]:
users_col.update_one({"name":"John"}, {"$set":{"name":"Will", "age":53}})

D - Delete

In [49]:
users_col.delete_one({"name":"Will"})

In [53]:
products_db = db_store.products

In [51]:
bag = {"name": "Bag", "price":23.78}

In [52]:
products_db.insert(bag)

/var/folders/1l/xv3wqjt14cgf1mmyz0bm3bnr0000gn/T/ipykernel_12209/1657952950.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  products_db.insert(bag)


ObjectId('668a889073c8b630e2131b6b')

In [55]:
phone = {"name":"iphone 11", "price": 800}
notebook = {"name":"Notebook", "price": 4.99}

In [56]:
products_db.insert_many([phone, notebook])

In [75]:
pen = {"name":"Pen2", "price":1.22}
insert_pen = products_db.insert_one(pen)

In [59]:
print(insert_pen.acknowledged)
print(insert_pen.inserted_id)

True
668a8a2173c8b630e2131b6f


In [60]:
water_bottle = {"name":"Water Bottle", "price": 14.5}
jar = {"name":"Jar", "price":4.99}
insert_several = products_db.insert_many([water_bottle, jar])
insert_several

In [61]:
print(insert_several.acknowledged)
print(insert_several.inserted_ids)


True
[ObjectId('668a8ac273c8b630e2131b70'), ObjectId('668a8ac273c8b630e2131b71')]


Error Handling When Inserting

In [67]:
try:
    products_db.insert_one(pen)
except errors.DuplicateKeyError:
    print("Duplicate Entry Detected")

Duplicate Entry Detected


In [69]:
try:
    products_db.insert_many([water_bottle, jar])
except errors.BulkWriteError:
    print("Bulk Error Detected")

Bulk Error Detected


In [70]:
mug = {"name":"Mug", "price":2.33}
try:
    products_db.insert_many([mug, jar])
except errors.BulkWriteError:
    print("Bulk Error Detected")

Bulk Error Detected


In [71]:
products_db

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'store'), 'products')

Find Functionality

In [72]:
found_pen = products_db.find_one({"name":"Pen"})

In [73]:
found_pen["_id"]

ObjectId('668a89d773c8b630e2131b6e')

In [76]:
# cursor
products_db.find({"name":"Pen2"})

In [79]:
pen_cursor = products_db.find({"name":"Pen2"})
for pen in pen_cursor:
    print(pen["_id"])

668a8a2173c8b630e2131b6f
668a8e6173c8b630e2131b73


In [81]:
pen_cursor.alive #cursor went through all results

False

In [94]:
pen_cursor = products_db.find({"name":"Pen2"})
pen_cursor.alive #cursor has not gone through all results

True

In [95]:
pen_cursor.next()

{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}

Sorting

In [105]:
for document in products_db.find({}).sort("price", ASCENDING):
    print(document)

{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}
{'_id': ObjectId('668a889073c8b630e2131b6b'), 'name': 'Bag', 'price': 23.78}
{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}


In [106]:
for document in products_db.find({}).sort("price", DESCENDING):
    print(document)

{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}
{'_id': ObjectId('668a889073c8b630e2131b6b'), 'name': 'Bag', 'price': 23.78}
{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}
{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}


In [101]:
ASCENDING

1

In [102]:
DESCENDING

-1

In [107]:
for document in products_db.find({}).sort("price", 1):
    print(document)

{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}
{'_id': ObjectId('668a889073c8b630e2131b6b'), 'name': 'Bag', 'price': 23.78}
{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}


In [108]:
for document in products_db.find({}).sort([("price", ASCENDING),("name", ASCENDING)]):
    print(document)

{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}
{'_id': ObjectId('668a889073c8b630e2131b6b'), 'name': 'Bag', 'price': 23.78}
{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}


In [109]:
for product in products_db.find({}).limit(5):
    print(product)

{'_id': ObjectId('668a889073c8b630e2131b6b'), 'name': 'Bag', 'price': 23.78}
{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}
{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}


In [110]:
for product in products_db.find({}).skip(5):
    print(product)

{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}


In [111]:
page_item_limit = 5
for page_num in [1, 2]:
    print("\n-- Page {} --".format(page_num))
    # Calculate the number of products to skip
    skip_amount = (page_num - 1) * page_item_limit
    # Find, skip, and limit the products for the current page
    for product in products_db.find({}).skip(skip_amount).limit(page_item_limit):
        print(product)


-- Page 1 --
{'_id': ObjectId('668a889073c8b630e2131b6b'), 'name': 'Bag', 'price': 23.78}
{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}
{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}

-- Page 2 --
{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}


Count documents

In [112]:
find_query_filter = {"name": "Pen2"}
find_cursor = products_db.find(find_query_filter)

In [113]:
find_cursor.count()

/var/folders/1l/xv3wqjt14cgf1mmyz0bm3bnr0000gn/T/ipykernel_12209/2083242281.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  find_cursor.count()


2

In [117]:
products_db.count_documents(find_query_filter)

2

In [118]:
find_cursor_skip = products_db.find(find_query_filter).skip(1)
products_db.count_documents(find_query_filter)

2

In [119]:
find_cursor_skip.count(with_limit_and_skip=True)

/var/folders/1l/xv3wqjt14cgf1mmyz0bm3bnr0000gn/T/ipykernel_12209/629907177.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  find_cursor_skip.count(with_limit_and_skip=True)


1

Distinct

In [121]:
product_cursor = products_db.find({})

In [124]:
distinct_count = product_cursor.distinct("name")
all_count = products_db.count_documents({})

print("{} unique product names found vs {} total entries".format(len(distinct_count), all_count))

8 unique product names found vs 9 total entries


Using projection

In [125]:
find_projection_cursor = products_db.find({"name":"Pen2"}, {"_id":0, "price":1})

In [126]:
for document in find_projection_cursor:
    print(document)

{'price': 1.22}
{'price': 1.22}


In [127]:
products_db.find_one({"name":"Pen2"}, {"_id":0, "price":1})

{'price': 1.22}

Comparision Operator

In [131]:
def print_cursor(cursor):
    for document in cursor:
        print(document)

In [132]:
lt_cursor = products_db.find({"price":{"$lt":2}})

In [133]:
print_cursor(lt_cursor)

{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}


In [135]:
lt_gte_cursor = products_db.find({"price":{"$lt": 10, "$gte": 2}})
print_cursor(lt_gte_cursor)

{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}


In [138]:
lt_ne_cursor = products_db.find({"price": {"$lt": 5}, "name": {"$ne": "Jar"}})
print_cursor(lt_ne_cursor)

{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}


Logical Operators

In [140]:
and_cursor = products_db.find(
    {"$and":[
        {"price":{"$lt":5}},
        {"name":{"$ne":"Mug"}}
    ]})
print_cursor(and_cursor)

{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}


In [141]:
or_cursor = products_db.find(
    {"$or":[
        {"price":{"$lt":5}},
        {"price":{"$gt":2}}
    ]})
print_cursor(or_cursor)

{'_id': ObjectId('668a889073c8b630e2131b6b'), 'name': 'Bag', 'price': 23.78}
{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}
{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a89d773c8b630e2131b6e'), 'name': 'Pen', 'price': 1.22}
{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22}
{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}
{'_id': ObjectId('668a8e6173c8b630e2131b73'), 'name': 'Pen2', 'price': 1.22}


In [144]:
nor_cursor = products_db.find(
    {"$nor":[
        {"price":{"$lt":2}},
        {"price":{"$gt":5}}
    ]})
print_cursor(nor_cursor)

{'_id': ObjectId('668a88e873c8b630e2131b6d'), 'name': 'Notebook', 'price': 4.99}
{'_id': ObjectId('668a8ac273c8b630e2131b71'), 'name': 'Jar', 'price': 4.99}
{'_id': ObjectId('668a8c9f73c8b630e2131b72'), 'name': 'Mug', 'price': 2.33}


In [147]:
type_cursor = products_db.find({"price":{"$type":"int"}})
print_cursor(type_cursor)

{'_id': ObjectId('668a88e873c8b630e2131b6c'), 'name': 'iphone 11', 'price': 800}


In [150]:
exists_cursor = products_db.find({"seller":{"$exists": True}})
print_cursor(exists_cursor)

{'_id': ObjectId('668a8a2173c8b630e2131b6f'), 'name': 'Pen2', 'price': 1.22, 'seller': 'Dan'}


In [156]:
regex_cursor = products_db.find({"name":{"$regex":"water", "$options":"i"}})
print_cursor(regex_cursor)

{'_id': ObjectId('668a8ac273c8b630e2131b70'), 'name': 'Water Bottle', 'price': 14.5}


Update Functionality

In [158]:
products_db.update({"name":"Pen"},{"$set":{"price": 3.22}})

/var/folders/1l/xv3wqjt14cgf1mmyz0bm3bnr0000gn/T/ipykernel_12209/1806000331.py:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  products_db.update({"name":"Pen"},{"$set":{"price": 3.22}})


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [161]:
bag_update = products_db.update_one({"name":"Bag"},{"$set":{"price": 24.38}})

In [163]:
print(bag_update.acknowledged)
print(bag_update.matched_count)
print(bag_update.modified_count)
print(bag_update.raw_result)
print(bag_update.upserted_id)

True
1
0
{'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}
None


In [164]:
replace_bag = products_db.replace_one({"name":"Bag"},{"product_name":"Bag", "price":28.77, "seller":"Dan"})

In [167]:
print(replace_bag.acknowledged)
print(replace_bag.matched_count)
print(replace_bag.modified_count)
print(replace_bag.raw_result)
print(replace_bag.upserted_id)

True
1
1
{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}
None


In [170]:
update_many_pro = products_db.update_many({"name":"Pen2"},{"$set": {"price": 3.5}})
update_many_pro

In [171]:
print(update_many_pro.acknowledged)
print(update_many_pro.matched_count)
print(update_many_pro.modified_count)
print(update_many_pro.raw_result)
print(update_many_pro.upserted_id)

True
2
2
{'n': 2, 'nModified': 2, 'ok': 1.0, 'updatedExisting': True}
None


In [174]:
update_upsert_res = products_db.update_many({"name":"Hat"},{"$set": {"price": 3.5}}, upsert=True)
update_upsert_res

In [175]:
print(update_upsert_res.acknowledged)
print(update_upsert_res.matched_count)
print(update_upsert_res.modified_count)
print(update_upsert_res.raw_result)
print(update_upsert_res.upserted_id)

True
0
0
{'n': 1, 'upserted': ObjectId('668ac9b9c68bee2ffe349b61'), 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}
668ac9b9c68bee2ffe349b61


In [177]:
products_db.update_many({}, {"$currentDate": {"date": True}})

In [178]:
#rename 
products_db.update_many({}, {"$rename": {"date": "dateCreated"}})

In [179]:
# price * 1.04
products_db.update_many({}, {"$mul": {"price": 1.04}})

In [180]:
# remove date
products_db.update_many({}, {"$unset": {"dateCreated": True}})

Delete Functionality

In [181]:
removes_product = products_db.remove({"name":"Pen"}, multi=True)

/var/folders/1l/xv3wqjt14cgf1mmyz0bm3bnr0000gn/T/ipykernel_12209/3772704908.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  removes_product = products_db.remove({"name":"Pen"}, multi=True)


In [182]:
delete_one_pro = products_db.delete_one({"name":"Pen2"})

In [183]:
print(delete_one_pro.acknowledged)
print(delete_one_pro.deleted_count)
print(delete_one_pro.raw_result)

True
1
{'n': 1, 'ok': 1.0}


In [185]:
delete_many_pro = products_db.delete_many({"name":"Pen2"})

Timestamp and datetime objects

In [187]:
time.time()

1720373183.436037

In [188]:
type(time.time())

float

In [189]:
datetime.now()

datetime.datetime(2024, 7, 7, 20, 26, 44, 316926)

In [191]:
datetime.now() + timedelta(days=21)

datetime.datetime(2024, 7, 28, 20, 27, 14, 528555)

In [192]:
datetime.utcnow()

/var/folders/1l/xv3wqjt14cgf1mmyz0bm3bnr0000gn/T/ipykernel_12209/3983769260.py:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime.utcnow()


datetime.datetime(2024, 7, 7, 17, 28, 8, 609832)

In [193]:
time.timezone

-7200

In [194]:
datetime.fromtimestamp(time.time())

datetime.datetime(2024, 7, 7, 20, 28, 40, 974987)

In [195]:
datetime.fromtimestamp(time.time() + time.timezone)

datetime.datetime(2024, 7, 7, 18, 29, 1, 730352)

Inserting datetime and timestamp entries

In [207]:
current_datetime = datetime.utcnow()  # Get the current UTC datetime
join_timestamp = time.time()  # Get the current timestamp
timezone_offset = -time.timezone 
document = {
    "name": "Ethan",
    "join_date_utc": current_datetime,
    "join_timestamp": join_timestamp,
    "subscription_end_date_utc": current_datetime + timedelta(days=31)
}
result = products_db.insert_one(document)
print(f"Inserted document ID: {result.inserted_id}")

Inserted document ID: 668ad43b73c8b630e2131b75


/var/folders/1l/xv3wqjt14cgf1mmyz0bm3bnr0000gn/T/ipykernel_12209/829422733.py:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_datetime = datetime.utcnow()  # Get the current UTC datetime


In [202]:
result=products_db.find_one({"join_date_utc": {"$lt": datetime(2025, 1, 1)}})
print(result)

{'_id': ObjectId('668ad2d473c8b630e2131b74'), 'name': 'Ethan', 'join_date_utc': datetime.datetime(2024, 7, 7, 17, 39, 32, 954000), 'join_timestamp': 1720373972.954709, 'subscription_end_date_utc': datetime.datetime(2024, 8, 7, 17, 39, 32, 954000)}


In [210]:
for document in result:
    print(document)
    print("\n")

TypeError: 'InsertOneResult' object is not iterable

BSON Timestamp

In [205]:
bson.Timestamp(int(time.time()), 1)

Timestamp(1720374307, 1)

In [209]:
products_db.insert_one({
    "name": "Frank",
    "join_date_utc": current_datetime,
    "join_timestamp": join_timestamp,
    "subscription_end_date_utc": current_datetime + timedelta(days=31)
})

In [211]:
products_db.find_one({"name":"Frank"})

{'_id': ObjectId('668ad45573c8b630e2131b77'),
 'name': 'Frank',
 'join_date_utc': datetime.datetime(2024, 7, 7, 17, 45, 31, 114000),
 'join_timestamp': 1720374331.114996,
 'subscription_end_date_utc': datetime.datetime(2024, 8, 7, 17, 45, 31, 114000)}

In [213]:
document1 = products_db.find_one({"name": "Frank"})
if document and "join_timestamp" in document:
    join_timestamp = document["join_timestamp"]
    
    # Convert the Unix timestamp to a datetime object
    join_datetime = datetime.fromtimestamp(join_timestamp)
    
    # Extract and print the time part of the datetime object
    print(join_datetime.time())
else:
    print("Document not found or 'join_timestamp' not present.")

20:45:31.114996


Converting UTC to local timezone

In [217]:
find_one_res = products_db.find_one({})
if products_db.count_documents({"name": "Ethan"}) == 0:
    # Insert a sample document with a join_date_utc field
    current_datetime_utc = datetime.utcnow()
    products_db.insert_one({
        "name": "Ethan",
        "join_date_utc": current_datetime_utc,
        "join_timestamp": time.time(),
        "subscription_end_date_utc": current_datetime_utc + timedelta(days=31)
    })

# Find one document from the collection
find_one_res = products_db.find_one({"name": "Ethan"})

# Check if the document contains the "join_date_utc" field and convert it to local timezone
if find_one_res and "join_date_utc" in find_one_res:
    join_date_utc = find_one_res["join_date_utc"]

    # Ensure the join_date_utc is timezone-aware
    if join_date_utc.tzinfo is None:
        join_date_utc = pytz.utc.localize(join_date_utc)

    # Convert the UTC datetime to local timezone
    local_timezone = pytz.timezone("America/New_York")  # Change this to your local timezone
    local_join_date = join_date_utc.astimezone(local_timezone)
    
    print(f"UTC Time: {join_date_utc}")
    print(f"Local Time: {local_join_date}")
else:
    print("Document not found or 'join_date_utc' field not present.")

UTC Time: 2024-07-07 17:39:32.954000+00:00
Local Time: 2024-07-07 13:39:32.954000-04:00


In [218]:
pytz.all_timezones

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/

Array

In [220]:
products_db.insert_many([
    {"name": "Bag", "departments": ["School"], "versions": [
        {"color": "Black", "size": "small", "qty": 5, "price": 17.79},
        {"color": "Red", "size": "small", "qty": 3, "price": 18.23},
        {"color": "Green", "size": "small", "qty": 5, "price": 20.03},
        {"color": "Black", "size": "large", "qty": 1, "price": 41.23},
        {"color": "Red", "size": "large", "qty": 10, "price": 46.82},
        {"color": "Green", "size": "large", "qty": 7, "price": 45.43},
    ]},
    {"name": "Mug", "departments": ["Home", "Kitchen"], "versions": [
        {"color": "White", "size": "11 oz", "qty": 14, "price": 14.79},
        {"color": "Blue", "size": "11 oz", "qty": 23, "price": 15.23},
        {"color": "Green", "size": "11 oz", "qty": 15, "price": 15.07},
        {"color": "White", "size": "15 oz", "qty": 11, "price": 24.43},
        {"color": "Blue", "size": "15 oz", "qty": 7, "price": 25.42},
        {"color": "Green", "size": "15 oz", "qty": 10, "price": 25.83},
    ]},
    {"name": "Pens", "departments": ["School", "Office"], "versions": [
        {"color": "Black", "type": "10 Pack", "qty": 40, "price": 14.79},
        {"color": "Red", "type": "10 Pack", "qty": 13, "price": 15.23},
        {"color": "Blue", "type": "10 Pack", "qty": 12, "price": 15.07}
    ]}
])

In [255]:
def list_docs(find_cursor):
    for document in find_cursor:
        print(document)

In [256]:
# search
find_cursor = products_db.find({"departments":"School"})
list_docs(find_cursor)

{'_id': ObjectId('668ad71573c8b630e2131b7b'), 'name': 'Bag', 'departments': ['School'], 'versions': [{'color': 'Black', 'size': 'small', 'qty': 5, 'price': 17.79}, {'color': 'Red', 'size': 'small', 'qty': 3, 'price': 18.23}, {'color': 'Green', 'size': 'small', 'qty': 5, 'price': 20.03}, {'color': 'Black', 'size': 'large', 'qty': 1, 'price': 41.23}, {'color': 'Red', 'size': 'large', 'qty': 10, 'price': 46.82}, {'color': 'Green', 'size': 'large', 'qty': 7, 'price': 45.43}]}
{'_id': ObjectId('668ad71573c8b630e2131b7d'), 'name': 'Pens', 'departments': ['School', 'Office'], 'versions': [{'color': 'Black', 'type': '10 Pack', 'qty': 40, 'price': 14.79}, {'color': 'Red', 'type': '10 Pack', 'qty': 13, 'price': 15.23}, {'color': 'Blue', 'type': '10 Pack', 'qty': 12, 'price': 15.07}]}


In [257]:
find_cursor = products_db.find({"departments":["Home", "Kitchen"]})
list_docs(find_cursor)

{'_id': ObjectId('668ad71573c8b630e2131b7c'), 'name': 'Mug', 'departments': ['Home', 'Kitchen'], 'versions': [{'color': 'White', 'size': '11 oz', 'qty': 14, 'price': 14.79}, {'color': 'Blue', 'size': '11 oz', 'qty': 23, 'price': 15.23}, {'color': 'Green', 'size': '11 oz', 'qty': 15, 'price': 15.07}, {'color': 'White', 'size': '15 oz', 'qty': 11, 'price': 24.43}, {'color': 'Blue', 'size': '15 oz', 'qty': 7, 'price': 25.42}, {'color': 'Green', 'size': '15 oz', 'qty': 10, 'price': 25.83}]}


In [258]:
find_cursor = products_db.find({"departments":{"$size":2}})
list_docs(find_cursor)

{'_id': ObjectId('668ad71573c8b630e2131b7c'), 'name': 'Mug', 'departments': ['Home', 'Kitchen'], 'versions': [{'color': 'White', 'size': '11 oz', 'qty': 14, 'price': 14.79}, {'color': 'Blue', 'size': '11 oz', 'qty': 23, 'price': 15.23}, {'color': 'Green', 'size': '11 oz', 'qty': 15, 'price': 15.07}, {'color': 'White', 'size': '15 oz', 'qty': 11, 'price': 24.43}, {'color': 'Blue', 'size': '15 oz', 'qty': 7, 'price': 25.42}, {'color': 'Green', 'size': '15 oz', 'qty': 10, 'price': 25.83}]}
{'_id': ObjectId('668ad71573c8b630e2131b7d'), 'name': 'Pens', 'departments': ['School', 'Office'], 'versions': [{'color': 'Black', 'type': '10 Pack', 'qty': 40, 'price': 14.79}, {'color': 'Red', 'type': '10 Pack', 'qty': 13, 'price': 15.23}, {'color': 'Blue', 'type': '10 Pack', 'qty': 12, 'price': 15.07}]}


In [259]:
find_cursor = products_db.find({"versions":{"$size":3}})
list_docs(find_cursor)

{'_id': ObjectId('668ad71573c8b630e2131b7d'), 'name': 'Pens', 'departments': ['School', 'Office'], 'versions': [{'color': 'Black', 'type': '10 Pack', 'qty': 40, 'price': 14.79}, {'color': 'Red', 'type': '10 Pack', 'qty': 13, 'price': 15.23}, {'color': 'Blue', 'type': '10 Pack', 'qty': 12, 'price': 15.07}]}


In [260]:
find_cursor = products_db.find({"versions": {"$exists": True}})

# Filter documents in Python based on 'versions' array length > 2
filtered_documents = [document for document in find_cursor if len(document.get('versions', [])) > 2]

# Iterate through the filtered documents and print each document
list_docs(find_cursor)

Arrays Of Embedded Documents

In [261]:
find_cursor = products_db.find({"versions.color":"Red"})
list_docs(find_cursor)

{'_id': ObjectId('668ad71573c8b630e2131b7b'), 'name': 'Bag', 'departments': ['School'], 'versions': [{'color': 'Black', 'size': 'small', 'qty': 5, 'price': 17.79}, {'color': 'Red', 'size': 'small', 'qty': 3, 'price': 18.23}, {'color': 'Green', 'size': 'small', 'qty': 5, 'price': 20.03}, {'color': 'Black', 'size': 'large', 'qty': 1, 'price': 41.23}, {'color': 'Red', 'size': 'large', 'qty': 10, 'price': 46.82}, {'color': 'Green', 'size': 'large', 'qty': 7, 'price': 45.43}]}
{'_id': ObjectId('668ad71573c8b630e2131b7d'), 'name': 'Pens', 'departments': ['School', 'Office'], 'versions': [{'color': 'Black', 'type': '10 Pack', 'qty': 40, 'price': 14.79}, {'color': 'Red', 'type': '10 Pack', 'qty': 13, 'price': 15.23}, {'color': 'Blue', 'type': '10 Pack', 'qty': 12, 'price': 15.07}]}


In [262]:
find_cursor = products_db.find({"versions.size":{"$exists":True}})
list_docs(find_cursor)

{'_id': ObjectId('668ad71573c8b630e2131b7b'), 'name': 'Bag', 'departments': ['School'], 'versions': [{'color': 'Black', 'size': 'small', 'qty': 5, 'price': 17.79}, {'color': 'Red', 'size': 'small', 'qty': 3, 'price': 18.23}, {'color': 'Green', 'size': 'small', 'qty': 5, 'price': 20.03}, {'color': 'Black', 'size': 'large', 'qty': 1, 'price': 41.23}, {'color': 'Red', 'size': 'large', 'qty': 10, 'price': 46.82}, {'color': 'Green', 'size': 'large', 'qty': 7, 'price': 45.43}]}
{'_id': ObjectId('668ad71573c8b630e2131b7c'), 'name': 'Mug', 'departments': ['Home', 'Kitchen'], 'versions': [{'color': 'White', 'size': '11 oz', 'qty': 14, 'price': 14.79}, {'color': 'Blue', 'size': '11 oz', 'qty': 23, 'price': 15.23}, {'color': 'Green', 'size': '11 oz', 'qty': 15, 'price': 15.07}, {'color': 'White', 'size': '15 oz', 'qty': 11, 'price': 24.43}, {'color': 'Blue', 'size': '15 oz', 'qty': 7, 'price': 25.42}, {'color': 'Green', 'size': '15 oz', 'qty': 10, 'price': 25.83}]}


In [263]:
find_cursor = products_db.find({"versions.size":{"$exists":True}})
list_docs(find_cursor)

{'_id': ObjectId('668ad71573c8b630e2131b7b'), 'name': 'Bag', 'departments': ['School'], 'versions': [{'color': 'Black', 'size': 'small', 'qty': 5, 'price': 17.79}, {'color': 'Red', 'size': 'small', 'qty': 3, 'price': 18.23}, {'color': 'Green', 'size': 'small', 'qty': 5, 'price': 20.03}, {'color': 'Black', 'size': 'large', 'qty': 1, 'price': 41.23}, {'color': 'Red', 'size': 'large', 'qty': 10, 'price': 46.82}, {'color': 'Green', 'size': 'large', 'qty': 7, 'price': 45.43}]}
{'_id': ObjectId('668ad71573c8b630e2131b7c'), 'name': 'Mug', 'departments': ['Home', 'Kitchen'], 'versions': [{'color': 'White', 'size': '11 oz', 'qty': 14, 'price': 14.79}, {'color': 'Blue', 'size': '11 oz', 'qty': 23, 'price': 15.23}, {'color': 'Green', 'size': '11 oz', 'qty': 15, 'price': 15.07}, {'color': 'White', 'size': '15 oz', 'qty': 11, 'price': 24.43}, {'color': 'Blue', 'size': '15 oz', 'qty': 7, 'price': 25.42}, {'color': 'Green', 'size': '15 oz', 'qty': 10, 'price': 25.83}]}


Updating Arrays

In [272]:
update_many_res = products_db.update_many({"name":"Hat"},{"$set":{"departments":["Office", "School"]}})
update_many_res.raw_result

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [274]:
pen2 = products_db.find_one({"name":"Hat"})["departments"]
print(pen2)

['Office', 'School']


In [280]:
update_many_res = products_db.update_many({"name":"Hat"},{"$pop":{"departments":1}})
hat = products_db.find_one({"name":"Hat"})["departments"]
print(hat)

['Office']


In [282]:
update_many_res = products_db.update_many({"name":"Hat"},{"$pull":{"departments":"Office"}})
hat = products_db.find_one({"name":"Hat"})["departments"]
print(hat)

[]


In [283]:
update_many_res = products_db.update_many({"name":"Hat"},{"$push":{"departments":"Office"}})
hat = products_db.find_one({"name":"Hat"})["departments"]
print(hat)

['Office']


Sets Vs Arrays

In [284]:
my_set = {"Home", "Office"}
print(type(my_set))
print(my_set)

<class 'set'>
{'Office', 'Home'}


In [285]:
my_set.add("School")
my_set

{'Home', 'Office', 'School'}

In [286]:
my_set.add("School")
my_set # set can not have 2 identical values

{'Home', 'Office', 'School'}

In [287]:
my_set.update(["Office", "Technology"])
my_set

{'Home', 'Office', 'School', 'Technology'}

In [288]:
update_many_res = products_db.update_many({"name":"Hat"}, {"$addToSet":{"departments":"Home"}})
update_many_res.raw_result

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [290]:
update_many_res = products_db.update_many({"name":"Hat"}, {"$addToSet":{"departments":{"$each":["Home","School"]}}})
update_many_res.raw_result

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}